In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
ls

'Colab Notebooks'/    open.zip                test_data.csv   train_data.csv
 data_loader_v2/      sample_submission.csv   test_x.csv
'My image model.tm'   test.csv                train.csv


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
index_col = 'index'
target_col = 'credit'

In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [ ]:
train['family_size']=train['family_size'].astype(int)
train['income_total']=train['income_total'].astype(int)
train['begin_month']=train['begin_month'].astype(int)
train['credit']=train['credit'].astype(int)

test['family_size']=test['family_size'].astype(int)
test['income_total']=test['income_total'].astype(int)
test['begin_month']=test['begin_month'].astype(int)


In [ ]:
train

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2,-6,1
1,1,F,N,Y,1,247500,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3,-5,1
2,2,M,Y,Y,0,450000,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2,-22,2
3,3,F,N,Y,0,202500,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2,-37,0
4,4,F,Y,Y,0,157500,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2,-26,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4,-2,1
26453,26453,F,N,Y,1,180000,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NaN,2,-47,2
26454,26454,F,Y,N,0,292500,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2,-25,2
26455,26455,M,N,Y,0,171000,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1,-59,2


In [ ]:
train=train.fillna('dontknow')
train=train.drop(labels='index',axis=1)

test=test.fillna('dontknow')
test=test.drop(labels='index',axis=1)

In [ ]:
test.isna().sum()

gender           0
car              0
reality          0
child_num        0
income_total     0
income_type      0
edu_type         0
family_type      0
house_type       0
DAYS_BIRTH       0
DAYS_EMPLOYED    0
FLAG_MOBIL       0
work_phone       0
phone            0
email            0
occyp_type       0
family_size      0
begin_month      0
dtype: int64

In [ ]:
!pip install category_encoders

In [ ]:
cat_features = [x for x in train.columns if train[x].dtype == 'object']
cont_features = [x for x in train.columns if x not in cat_features + [target_col]]
feature_cols = cont_features + cat_features
print(len(feature_cols), len(cat_features), len(cont_features))

18 8 10


In [ ]:
target=train['credit']

In [ ]:
from category_encoders import LeaveOneOutEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import CatBoostEncoder
from sklearn.preprocessing import OrdinalEncoder

xgb_cat_features = []
lgb_cat_features = []
cb_cat_features = []
ridge_cat_features = []
sgd_cat_features = []
hgbc_cat_features = []

loo_features = []
le_features = []
CBE_features=[]
Ord_features=[]

In [ ]:
def label_encode(train_df, test_df, column):
    le = LabelEncoder()
    new_feature = "{}_le".format(column)
    le.fit(train_df[column].unique().tolist() + test_df[column].unique().tolist())
    train_df[new_feature] = le.transform(train_df[column])
    test_df[new_feature] = le.transform(test_df[column])
    return new_feature

def ordinal_encode(train_df, test_df, column):
    Ord = OrdinalEncoder()
    new_feature = "{}_ord".format(column)
    Ord.fit(train_df[column].unique().tolist() + test_df[column].unique().tolist())
    train_df[new_feature] = Ord.transform(train_df[column])
    test_df[new_feature] = Ord.transform(test_df[column])
    return new_feature
    
def loo_encode(train_df, test_df, column):
    loo = LeaveOneOutEncoder()
    new_feature = "{}_loo".format(column)
    loo.fit(train_df[column], train_df["credit"])
    train_df[new_feature] = loo.transform(train_df[column])
    test_df[new_feature] = loo.transform(test_df[column])
    return new_feature

def CBE(train_df, test_df, column):
    CBE=CatBoostEncoder()
    new_feature="{}_CBE".format(column)
    CBE.fit(train_df[column],train_df["credit"])
    train_df[new_feature]=CBE.transform(train_df[column])
    test_df[new_feature]=CBE.transform(test_df[column])
    return new_feature

In [ ]:
for feature in cat_features:
    loo_features.append(loo_encode(train, test, feature))
    #le_features.append(label_encode(train, test, feature))
    CBE_features.append(CBE(train,test,feature))

In [ ]:
test.isna().sum()

gender             0
car                0
reality            0
child_num          0
income_total       0
income_type        0
edu_type           0
family_type        0
house_type         0
DAYS_BIRTH         0
DAYS_EMPLOYED      0
FLAG_MOBIL         0
work_phone         0
phone              0
email              0
occyp_type         0
family_size        0
begin_month        0
gender_loo         0
gender_CBE         0
car_loo            0
car_CBE            0
reality_loo        0
reality_CBE        0
income_type_loo    0
income_type_CBE    0
edu_type_loo       0
edu_type_CBE       0
family_type_loo    0
family_type_CBE    0
house_type_loo     0
house_type_CBE     0
occyp_type_loo     0
occyp_type_CBE     0
dtype: int64



---



In [ ]:
xgb_cat_features.extend(loo_features)
lgb_cat_features.extend(loo_features)
cb_cat_features.extend(cat_features)
ridge_cat_features.extend(loo_features)
sgd_cat_features.extend(loo_features)
hgbc_cat_features.extend(loo_features)

In [ ]:
!pip install catboost

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import log_loss

In [ ]:
random_state = 2021
n_folds = 10
k_fold = StratifiedKFold(n_splits=n_folds, random_state=random_state, shuffle=True)

In [ ]:
xgb_train_preds = np.zeros(len(train.index), )
xgb_test_preds = np.zeros(len(test.index), )
xgb_features = xgb_cat_features + cont_features

lgb_train_preds = np.zeros(len(train.index), )
lgb_test_preds = np.zeros(len(test.index), )
lgb_features = lgb_cat_features + cont_features

cb_train_preds = np.zeros(len(train.index), )
cb_test_preds = np.zeros(len(test.index), )
cb_features = cb_cat_features + cont_features

ridge_train_preds = np.zeros(len(train.index), )
ridge_test_preds = np.zeros(len(test.index), )
ridge_features = ridge_cat_features + cont_features

sgd_train_preds = np.zeros(len(train.index), )
sgd_test_preds = np.zeros(len(test.index), )
sgd_features = sgd_cat_features + cont_features

hgbc_train_preds = np.zeros(len(train.index), )
hgbc_test_preds = np.zeros(len(test.index), )
hgbc_features = hgbc_cat_features + cont_features

In [ ]:
lgb_features

['gender_loo',
 'car_loo',
 'reality_loo',
 'income_type_loo',
 'edu_type_loo',
 'family_type_loo',
 'house_type_loo',
 'occyp_type_loo',
 'child_num',
 'income_total',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'work_phone',
 'phone',
 'email',
 'family_size',
 'begin_month']

In [ ]:
l1_train = pd.DataFrame(data={
    "xgb": xgb_train_preds.tolist(),
    "lgb": lgb_train_preds.tolist(),
    "cb": cb_train_preds.tolist(),
    "ridge": ridge_train_preds.tolist(),
    "sgd": sgd_train_preds.tolist(),
    "hgbc": hgbc_train_preds.tolist(),
    "target": target.tolist()
})
l1_test = pd.DataFrame(data={
    "xgb": xgb_test_preds.tolist(),
    "lgb": lgb_test_preds.tolist(),
    "cb": cb_test_preds.tolist(),
    "sgd": sgd_test_preds.tolist(),
    "ridge": ridge_test_preds.tolist(),    
    "hgbc": hgbc_test_preds.tolist(),
})

In [ ]:
l1_train

,xgb,lgb,cb,ridge,sgd,hgbc,target
0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...
26452,0.0,0.0,0.0,0.0,0.0,0.0,1
26453,0.0,0.0,0.0,0.0,0.0,0.0,2
26454,0.0,0.0,0.0,0.0,0.0,0.0,2
26455,0.0,0.0,0.0,0.0,0.0,0.0,2


In [ ]:

train_preds = np.zeros(len(l1_train.index), )
test_preds = np.zeros(len(l1_test.index), )
features = ["xgb", "lgb", "cb", "ridge", "sgd", "hgbc"]
features

['xgb', 'lgb', 'cb', 'ridge', 'sgd', 'hgbc']

In [ ]:
import pandas

In [ ]:
train_preds.shape

(26457,)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
for fold, (train_index, test_index) in enumerate(k_fold.split(train, target)):
    print("--> Fold {}".format(fold + 1))
    y_train = target.iloc[train_index]
    y_valid = target.iloc[test_index]

    xgb_x_train = pandas.DataFrame(train[xgb_features].iloc[train_index])
    xgb_x_valid = pandas.DataFrame(train[xgb_features].iloc[test_index])

    lgb_x_train = pandas.DataFrame(train[lgb_features].iloc[train_index])
    lgb_x_valid = pandas.DataFrame(train[lgb_features].iloc[test_index])

    cb_x_train = pandas.DataFrame(train[cb_features].iloc[train_index])
    cb_x_valid = pandas.DataFrame(train[cb_features].iloc[test_index])

    ridge_x_train = pandas.DataFrame(train[ridge_features].iloc[train_index])
    ridge_x_valid = pandas.DataFrame(train[ridge_features].iloc[test_index])

    sgd_x_train = pandas.DataFrame(train[sgd_features].iloc[train_index])
    sgd_x_valid = pandas.DataFrame(train[sgd_features].iloc[test_index])

    hgbc_x_train = pandas.DataFrame(train[hgbc_features].iloc[train_index])
    hgbc_x_valid = pandas.DataFrame(train[hgbc_features].iloc[test_index])

    xgb_model = XGBClassifier(
        seed=random_state,
        n_estimators=10000,
        verbosity=1,
        eval_metric="mlogloss",
        tree_method="gpu_hist",
        gpu_id=0,
        alpha=7.105038963844129,
        colsample_bytree=0.25505629740052566,
        gamma=0.4999381950212869,
        reg_lambda=1.7256912198205319,
        learning_rate=0.011823142071967673,
        max_bin=338,
        max_depth=8,
        min_child_weight=2.286836198630466,
        subsample=0.618417952155855,
        objective="multi:softprob"
    )
  

    xgb_model.fit(
        xgb_x_train,
        y_train,
        eval_set=[(xgb_x_valid, y_valid)], 
        verbose=0,
        early_stopping_rounds=200
    )

    train_oof_preds = xgb_model.predict_proba(xgb_x_valid)[:,1]
    test_oof_preds = xgb_model.predict_proba(test[xgb_features])[:,1]
    xgb_train_preds[test_index] = train_oof_preds
    xgb_test_preds += test_oof_preds / n_folds
    print(": XGB - ROC AUC Score = {}".format(accuracy_score(y_valid, xgb_model.predict(xgb_x_valid))))

    lgb_model = LGBMClassifier(
        cat_feature=[x for x in range(len(lgb_cat_features))],
        random_state=random_state,
        cat_l2=25.999876242730252,
        cat_smooth=89.2699690675538,
        colsample_bytree=0.2557260109926193,
        early_stopping_round=200,
        learning_rate=0.00918685483594994,
        max_bin=788,
        max_depth=81,
        metric="multi_logloss",
        min_child_samples=292,
        min_data_per_group=177,
        n_estimators=1600000,
        n_jobs=-1,
        num_leaves=171,
        reg_alpha=0.7115353581785044,
        reg_lambda=5.658115293998945,
        subsample=0.9262904583735796,
        subsample_freq=1,
        verbose=-1,
        objective="multiclass"
    )
    lgb_model.fit(
        lgb_x_train,
        y_train,
        eval_set=[(lgb_x_valid, y_valid)], 
        verbose=0,
    )

    train_oof_preds = lgb_model.predict_proba(lgb_x_valid)[:,1]
    test_oof_preds = lgb_model.predict_proba(test[lgb_features])[:,1]
    lgb_train_preds[test_index] = train_oof_preds
    lgb_test_preds += test_oof_preds / n_folds
    print(": LGB - ROC AUC Score = {}".format(accuracy_score(y_valid, lgb_model.predict(lgb_x_valid))))

    cb_model = CatBoostClassifier(
        verbose=0,
        loss_function="MultiClass",
        random_state=random_state,
        num_boost_round=20000,
        od_type="Iter",
        od_wait=200,
        task_type="GPU",
        devices="0",
        cat_features=[x for x in range(len(cb_cat_features))],
        bagging_temperature=1.288692494969795,
        grow_policy="Depthwise",
        l2_leaf_reg=9.847870133539244,
        learning_rate=0.01877982653902465,
        max_depth=8,
        min_data_in_leaf=1,
        penalties_coefficient=2.1176668909602734,
    )
    cb_model.fit(
        cb_x_train,
        y_train,
        eval_set=[(cb_x_valid, y_valid)], 
        verbose=0,
    )

    train_oof_preds = cb_model.predict_proba(cb_x_valid)[:,1]
    test_oof_preds = cb_model.predict_proba(test[cb_features])[:,1]
    cb_train_preds[test_index] = train_oof_preds
    cb_test_preds += test_oof_preds / n_folds
    print(": CB - ROC AUC Score = {}".format(accuracy_score(y_valid, cb_model.predict(cb_x_valid))))
    
    ridge_model = CalibratedClassifierCV(
        RidgeClassifier(random_state=random_state),
        cv=3,
    )
    ridge_model.fit(
        ridge_x_train,
        y_train,
    )

    train_oof_preds = ridge_model.predict_proba(ridge_x_valid)[:,-1]
    test_oof_preds = ridge_model.predict_proba(test[ridge_features])[:,-1]
    ridge_train_preds[test_index] = train_oof_preds
    ridge_test_preds += test_oof_preds / n_folds
    print(": Ridge - ROC AUC Score = {}".format(accuracy_score(y_valid, ridge_model.predict(ridge_x_valid))))
    
    sgd_model = CalibratedClassifierCV(
        SGDClassifier(
            random_state=random_state,
            n_jobs=-1,
            loss="squared_hinge",
        ),
        cv=3,
    )
    sgd_model.fit(
        sgd_x_train,
        y_train,
    )

    train_oof_preds = sgd_model.predict_proba(sgd_x_valid)[:,-1]
    test_oof_preds = sgd_model.predict_proba(test[sgd_features])[:,-1]
    sgd_train_preds[test_index] = train_oof_preds
    sgd_test_preds += test_oof_preds / n_folds
    print(": SGD - ROC AUC Score = {}".format(accuracy_score(y_valid, sgd_model.predict(sgd_x_valid))))
    
    hgbc_model = HistGradientBoostingClassifier(
        l2_regularization=1.766059063693552,
        learning_rate=0.10675193678150449,
        max_bins=128,
        max_depth=31,
        max_leaf_nodes=185,
        random_state=2021
    )
    hgbc_model.fit(
        hgbc_x_train,
        y_train,
    )

    train_oof_preds = hgbc_model.predict_proba(hgbc_x_valid)[:,-1]
    test_oof_preds = hgbc_model.predict_proba(test[hgbc_features])[:,-1]
    hgbc_train_preds[test_index] = train_oof_preds
    hgbc_test_preds += test_oof_preds / n_folds
    print(": HGBC - ROC AUC Score = {}".format(accuracy_score(y_valid, hgbc_model.predict(hgbc_x_valid))))
    print("")
    
#print("--> Overall metrics")
#print(": XGB - ROC AUC Score = {}".format(accuracy_score(target, xgb_train_preds)))
#print(": LGB - ROC AUC Score = {}".format(accuracy_score(target, lgb_train_preds)))
#print(": CB - ROC AUC Score = {}".format(accuracy_score(target, cb_train_preds)))
#print(": Ridge - ROC AUC Score = {}".format(accuracy_score(target, ridge_train_preds)))
#print(": SGD - ROC AUC Score = {}".format(accuracy_score(target, sgd_train_preds)))
#print(": HGBC - ROC AUC Score = {}".format(accuracy_score(target, hgbc_train_preds)))

--> Fold 1
: XGB - ROC AUC Score = 0.7195767195767195
: LGB - ROC AUC Score = 0.7037037037037037
: CB - ROC AUC Score = 0.7142857142857143
: Ridge - ROC AUC Score = 0.6424792139077853
: SGD - ROC AUC Score = 0.6413454270597128
: HGBC - ROC AUC Score = 0.7097505668934241

--> Fold 2
: XGB - ROC AUC Score = 0.7226001511715797
: LGB - ROC AUC Score = 0.7033257747543462
: CB - ROC AUC Score = 0.7154195011337868
: Ridge - ROC AUC Score = 0.6413454270597128
: SGD - ROC AUC Score = 0.6413454270597128
: HGBC - ROC AUC Score = 0.7120181405895691

--> Fold 3
: XGB - ROC AUC Score = 0.718442932728647
: LGB - ROC AUC Score = 0.6969009826152683
: CB - ROC AUC Score = 0.7142857142857143
: Ridge - ROC AUC Score = 0.6428571428571429
: SGD - ROC AUC Score = 0.6413454270597128
: HGBC - ROC AUC Score = 0.7161753590325018

--> Fold 4
: XGB - ROC AUC Score = 0.7282690854119426
: LGB - ROC AUC Score = 0.7120181405895691
: CB - ROC AUC Score = 0.72713529856387
: Ridge - ROC AUC Score = 0.6421012849584278
: S

In [ ]:
from scipy.special import expit
from sklearn.calibration import CalibratedClassifierCV

random_state = 2021
n_folds = 10
k_fold = StratifiedKFold(n_splits=n_folds, random_state=random_state, shuffle=True)

l1_train = pandas.DataFrame(data={
    "xgb": xgb_train_preds.tolist(),
    "lgb": lgb_train_preds.tolist(),
    "cb": cb_train_preds.tolist(),
    "ridge": ridge_train_preds.tolist(),
    "sgd": sgd_train_preds.tolist(),
    "hgbc": hgbc_train_preds.tolist(),
    "target": target.tolist()
})
l1_test = pandas.DataFrame(data={
    "xgb": xgb_test_preds.tolist(),
    "lgb": lgb_test_preds.tolist(),
    "cb": cb_test_preds.tolist(),
    "sgd": sgd_test_preds.tolist(),
    "ridge": ridge_test_preds.tolist(),    
    "hgbc": hgbc_test_preds.tolist(),
})

train_preds = np.zeros(len(l1_train.index), )
test_preds = np.zeros(len(l1_test.index), )
features = ["xgb", "lgb", "cb", "ridge", "sgd", "hgbc"]

for fold, (train_index, test_index) in enumerate(k_fold.split(l1_train, target)):
    print("--> Fold {}".format(fold + 1))
    y_train = target.iloc[train_index]
    y_valid = target.iloc[test_index]

    x_train = pandas.DataFrame(l1_train[features].iloc[train_index])
    x_valid = pandas.DataFrame(l1_train[features].iloc[test_index])
    
    model = CalibratedClassifierCV(
        RidgeClassifier(random_state=random_state), 
        cv=3
    )
    model.fit(
        x_train,
        y_train,
    )

    train_oof_preds = model.predict_proba(x_valid)
    test_oof_preds = model.predict_proba(l1_test[features])
    #train_preds[test_index] = train_oof_preds
    #test_preds += test_oof_preds / n_folds
    print(": ROC AUC Score = {}".format(accuracy_score(y_valid,model.predict(x_valid))))
    print("")
    
#print("--> Overall metrics")
#print(": ROC AUC Score = {}".format(accuracy_score(target, model.predict(x_train))))

--> Fold 1
: ROC AUC Score = 0.7150415721844293

--> Fold 2
: ROC AUC Score = 0.7207105064247922

--> Fold 3
: ROC AUC Score = 0.7150415721844293

--> Fold 4
: ROC AUC Score = 0.7294028722600151

--> Fold 5
: ROC AUC Score = 0.7173091458805745

--> Fold 6
: ROC AUC Score = 0.708994708994709

--> Fold 7
: ROC AUC Score = 0.7120181405895691

--> Fold 8
: ROC AUC Score = 0.7187145557655955

--> Fold 9
: ROC AUC Score = 0.7164461247637051

--> Fold 10
: ROC AUC Score = 0.7122873345935727



In [ ]:
submit=pd.read_csv("sample_submission.csv",index_col=index_col)

In [ ]:
submit

,0,1,2
index,,,
26457,0,0,0
26458,0,0,0
26459,0,0,0
26460,0,0,0
26461,0,0,0
...,...,...,...
36452,0,0,0
36453,0,0,0
36454,0,0,0


In [ ]:
submit[submit.columns]=test_oof_preds

In [ ]:
submit

,0,1,2
index,,,
26457,0.077448,0.099250,0.823302
26458,0.166222,0.175642,0.658137
26459,0.076271,0.117439,0.806289
26460,0.076430,0.151185,0.772385
26461,0.078268,0.183198,0.738534
...,...,...,...
36452,0.151613,0.252988,0.595399
36453,0.201683,0.361396,0.436921
36454,0.075653,0.101977,0.822370


In [ ]:
submit.to_csv("submit.csv")